In [9]:
import pandas as pd
import numpy as np

In [10]:
gcs_images_prefix = 'gs://kolban-fashion-products/fashion-dataset/images'
product_set_id    = 'my_product_set'

In [11]:
# Define a function which takes a data frame and writes that as a Product Search representation CSV
# Inputs:
# data: The data to write
# fileName: The file to write
def writeProductSearchCSV(data, fileName):
    new = pd.DataFrame()
    new['image-uri']            = gcs_images_prefix + "/" + data['id'].astype(str) + ".jpg"
    new['image-id']             = ''
    new['product-set-id']       = product_set_id
    new['product-id']           = data['id']
    new['product-category']     = 'apparel-v2'
    new['product-display-name'] = data['productDisplayName']
    new['labels']               = ''
    new['bounding-poly']        = ''
    new.to_csv(fileName, index=False)

In [12]:
data = pd.read_csv('styles.csv',error_bad_lines=False)
# We truncate down to 19500 entries because the CSV can only contain 20,000 entries at one time.
# Test with only 2000 entries
size   = len(data.index)
offset = 0
batch  = 10000
while offset < size:
    toValue = offset+batch-1 if (offset+batch-1) < size else size
    print("Writing from {} to {}".format(offset, toValue))

    current = data.truncate(before=offset, after=toValue)
    #print(current)
    writeProductSearchCSV(current, "{}_{}.csv".format(offset, toValue));
    offset = offset + batch


b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 35748: expected 10 fields, saw 11\nSkipping line 35962: expected 10 fields, saw 11\nSkipping line 37770: expected 10 fields, saw 11\nSkipping line 38105: expected 10 fields, saw 11\nSkipping line 38275: ex

The format of Vision API Product Search is:

* image-uri
* image-id*
* product-set-id
* product-id
* product-category
* product-display-name*
* labels*
* bounding-poly*


With the CSV now created and the images uploaded to a bucket, we can now perform the import of the information into Vision API Product Search.  One way to achieve this is to use `gcloud`.

For example:

```
gcloud beta ml vision product-search product-sets import 
```

!gsutil cp new.csv {gcs_prefix}

gsutil cp *.csv gs://kolban-fashion-products